In [ ]:
from scipy.integrate import quad,dblquad,nquad
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt

# Potentials


In [ ]:
##Calculation of toroidal potentials

Nphi = 24  #number of fluxes, always a global variable
N = 8 #number of electrons
ly = 4 #y-circumference, 
#alpha = 1 #aspect ratio Ly/Lx
#ly = np.sqrt(2*np.pi*Nphi*alpha) #Ly in terms of aspect ratio
Pi = np.pi

#Coulomb potential on an actual physical torus
def Vtorus(x,y,Ly,Nphi):
    Lx = Nphi*2*Pi/Ly
    theta = 2*Pi*x/Lx
    phi = 2*Pi*y/Ly
    return 1/np.sqrt(Lx**2/(2*Pi**2)*(1-np.cos(theta))+Ly**2/(2*Pi**2)*(1-np.cos(theta)*np.cos(phi))+Lx*Ly/(2*Pi**2)*(1+np.cos(phi)-np.cos(theta)-np.cos(theta)*np.cos(phi)))

#Fourier series coefficients on the y direction
def FTy(q,x):
    def f(y,X):
        return Vtorus(X,y,ly,Nphi)*np.cos(2*Pi*q*y/ly)#(1/np.sqrt(X**2+y**2)+1/np.sqrt(X**2+(y-ly)**2))*
    I1, err = quad(f,0,ly,args=(x,))
    #I2, err = quad(f,0,ly,args=(Lx-x,))
    return I1#+I2

#Integrate the fourier series over x
def TorusIntFTy(m,k):
    inf = Nphi*2*Pi/ly 
    eps = 0
    dmax = 1 #number of periodic copies that we integrate over, dmax = 0 is valid for small ly only
    def fdir(x): #contains both direct and exchange parts
        dsum = 0
        for d in range(-dmax,dmax+1):
            dsum = dsum + np.exp(-(x)**2/2)*FTy(m-d*Nphi,x-2*Pi*k/ly)*np.exp(-2*Pi**2*(m-d*Nphi)**2/ly**2)-np.exp(-(x-2*Pi*(m+k)/ly)**2/2)*FTy(k-d*Nphi,x-2*Pi*k/ly)*np.exp(-2*Pi**2*(k-d*Nphi)**2/ly**2)#*np.exp(-2*Pi**2*(k**2-2*d*Nphi*(k-m)-m**2)/ly**2)#**np.exp(2*Pi**2*2*d*Nphi*(k-m)2*d*Nphi*(k-m)-/ly**2)
        return dsum
    dpoints = [i*inf+2*Pi*k/ly for i in range(-dmax-1,dmax+1)]
    I1, err = quad(fdir,-(dmax+1)*inf,0,points=([i*inf+2*Pi*k/ly for i in range(-dmax-1,0)],))
    I2, err = quad(fdir,0,(dmax+1)*inf,points=([i*inf+2*Pi*k/ly for i in range(0,dmax+1)],))
    #I1m, err = quad(fdir,-inf,0,points=(-inf+2*Pi*k/ly,))
    return 1/np.sqrt(2*np.pi)*(I1+I2)/ly#*np.exp(-2*Pi**2*m**2/ly**2)#+I1m)#-I2-I2m

K = np.arange(0,Nphi//2)
#Vk0=np.zeros(Nphi//2)
#Vk1=np.zeros(Nphi//2)
VkAll = np.zeros([Nphi//2,Nphi//2])

#for i in K:
#    Vk0[i] = TorusIntFTy(0,i+1)
#    Vk1[i] = TorusIntFTy(1,i+1)
#for i in K:
#    for j in range(i,Nphi//2):
#       VkAll[i,j] = TorusIntFTy(i,j+1)
plt.scatter(2*Pi*K/ly,VkAllHald[0,0:len(K)])
plt.scatter(2*Pi*K/ly,VkAllHald[1,0:len(K)])
plt.scatter(2*Pi*K/ly,VkAllHald[2,0:len(K)])
plt.xlim(0,6)
#plt.scatter(K,VkAll_12[3,0:len(K)])
#plt.scatter(K,VkAll_12[4,0:len(K)])#print(Vk0)
#print(Vk1)
#VkAll

In [ ]:
#Precomputed potentials

#cylinder for ly = 4
VK0_4 = np.array([0.596776, 0.33901, 0.218187, 0.161551, 0.128503, 0.106759, 0.0913416,
                  0.0798301, 0.0709033, 0.0637765, 0.0579541, 0.0531076, 0.0490102, 0.0455005, 
                  0.0424603, 0.0398014, 0.0374561, 0.035372, 0.0335077,
                  0.0318303, 0.0303129, 0.0289336, 0.0276744, 0.0265203])
VK0_4reg = np.array([-0.11832, -0.079075, -0.021867, -0.00786881, -0.00377062, -0.0021099,
                     -0.00130293, -0.00086209, -0.000600406, -0.000435098, -0.000325468,
                     -0.00024986, -0.00019602, -0.000156627, -0.000127136, -0.000104617,
                     -0.0000871229, -0.0000733206, -0.0000622988, -0.0000533773,
                     -0.0000460833, -0.0000400533, -0.000035044, -0.0000308307])
VK1_4 = np.array([0., 0.00356477, 0.000735286, 0.000272085, 0.000132284, 0.0000745624,
                  0.0000462383, 0.0000306743, 0.0000214012, 0.0000155284, 0.0000116264, 
                  8.93191e-6, 7.01095e-6, 5.60435e-6, 4.55066e-6, 3.74567e-6, 
                  3.12006e-6, 2.62648e-6, 2.23184e-6, 1.91251e-6, 
                  1.65135e-6, 1.43568e-6, 1.25601e-6, 1.10513e-6])
U00_4 = -0.833847
#cylinder for ly = 5.49 (alpha = 0.2)
VkAll_549 = np.zeros([12,12])
VkAll_549[0,:] = np.array([0.563283, 0.43325, 0.295961, 0.220709, 0.175957, 0.146336, 0.125274, 0.109524, 0.0972984, 0.0875324, 0.0795504, 0.0729041])
VkAll_549[1,:] = np.array([0., 0.0217581, 0.00698783, 0.00252845, 0.00119676, 0.000665547, 0.000409589, 0.000270429, 0.000188076, 0.000136159, 0.000101779, 0.0000780943])
VkAll_549[2,:] = np.array([0., 0., 0.000108596, 0.000016303, 3.90379e-6, 1.36033e-6, 5.82657e-7, 2.85067e-7, 1.53327e-7, 8.85841e-8, 5.41373e-8, 3.46236e-8])
VkAll_549[3,:] = np.array([0., 0., 0., 5.20143e-8, 4.88546e-9, 9.46789e-10, 2.69695e-10, 9.54962e-11, 3.91608e-11, 1.78862e-11, 8.87816e-12, 4.70995e-12])
#cylinder for ly = 6.14 (alpha = 0.25)
VkAll_614 = np.zeros([12,12])
VkAll_614[0,:] = np.array([0.534226, 0.45347, 0.324029, 0.244296, 0.195561, 0.162978, 0.139686, 0.122214, 0.108626, 0.0977569, 0.0888651, 0.0814561])
VkAll_614[1,:] = np.array([0., 0.0323951, 0.0131468, 0.00505428, 0.00239993, 0.00133334, 0.000819807, 0.000540906, 0.000376002, 0.000272113, 0.000203349, 0.000155996])
VkAll_614[2,:] = np.array([0., 0., 0.000416791, 0.0000809119, 0.0000193137, 6.56932e-6, 2.77708e-6, 1.34828e-6, 7.21616e-7, 4.15502e-7, 2.53315e-7, 1.61716e-7])
VkAll_614[3,:] = np.array([0., 0., 0., 7.98973e-7, 8.49777e-8, 1.50386e-8, 4.08871e-9, 1.41357e-9, 5.71416e-10, 2.58534e-10, 1.27484e-10, 6.73056e-11])
#cylinder for ly = 6.72 (alpha = 0.3)
VkAll_672 = np.zeros([12,12])
VkAll_672[0,:] = np.array([0.50721, 0.462341, 0.344757, 0.263491, 0.212085, 0.177269, 0.152197, 0.133306, 0.118572, 0.106763, 0.0970888, 0.0890197])
VkAll_672[1,:] = np.array([0., 0.0415163, 0.0202852, 0.0084474, 0.00407727, 0.00227437, 0.0014006, 0.000924809, 0.000643137, 0.000465558, 0.000347969, 0.000266971])
VkAll_672[2,:] = np.array([0., 0., 0.0010114, 0.000251883, 0.0000636295, 0.0000214935, 9.01738e-6, 4.35654e-6, 2.32409e-6, 1.33516e-6, 8.12661e-7, 5.18166e-7])
VkAll_672[3,:] = np.array([0., 0., 0., 5.04182e-6, 6.77651e-7, 1.17737e-7, 3.08494e-8, 1.04502e-8, 4.17327e-9, 1.87321e-9, 9.18581e-10, 4.83018e-10])
#cylinder for ly =7.265 (alpha = 0.35)
VkAll_726 = np.zeros([12,12])
VkAll_726[0,:] = np.array([0.482293, 0.464477, 0.360259, 0.279615, 0.226541, 0.190054, 0.163543, 0.143454, 0.127724, 0.115085, 0.104711, 0.0960454])
VkAll_726[1,:] = np.array([0., 0.049175, 0.027932, 0.0126677, 0.00628117, 0.00353522, 0.00218613, 0.00144678, 0.00100752, 0.000729988, 0.000545955, 0.000419063])
VkAll_726[2,:] = np.array([0., 0., 0.00191991, 0.000595816, 0.00016481, 0.0000563782, 0.0000236336, 0.0000113985, 6.07206e-6, 3.4845e-6, 2.11909e-6, 1.35029e-6])
VkAll_726[3,:] = np.array([0., 0., 0., 0.0000195168, 3.37252e-6, 6.15924e-7, 1.5846e-7, 5.28672e-8, 2.091e-8, 9.32576e-9, 4.55269e-9, 2.38611e-9])
#cylinder for ly =7.77 (alpha = 0.4)
VkAll_777 = np.zeros([12,12])
VkAll_777[0,:] = np.array([0.460173, 0.462378, 0.371247, 0.292738, 0.238859, 0.20122, 0.173612, 0.152552, 0.13599, 0.122638, 0.111654, 0.102463])
VkAll_777[1,:] = np.array([0. , 0.0552623, 0.0353889, 0.0173987, 0.00891696, 0.00508387, 0.00316423, 0.00210189, 0.00146713, 0.00106465, 0.000797114, 0.000612336])
VkAll_777[2,:] = np.array([0., 0., 0.00307353, 0.00114531, 0.000350641, 0.000123485, 0.0000521079, 0.0000251806, 0.0000134227, 7.7044e-6, 4.68568e-6, 2.9857e-6])
VkAll_777[3,:] = np.array([0., 0., 0., 0.0000534638, 0.0000115937, 2.31827e-6, 5.99842e-7, 1.98686e-7, 7.81167e-8, 3.46906e-8, 1.68828e-8, 8.82799e-9])
#cyllinder for ly = 8
VK0_8 = np.array([0.45048, 0.460404, 0.375232, 0.298115, 0.244097, 0.206061, 0.178033,
                  0.156583, 0.139672, 0.126017, 0.11477, 0.105351, 0.097351, 0.0904736,
                  0.0844993, 0.079262, 0.0746336, 0.0705143, 0.0668246, 0.0635009, 
                  0.0604914, 0.0577536, 0.0552525, 0.0529586])
VK1_8 = np.array([0, 0.0576957, 0.0388001, 0.019777, 0.0103095, 0.00592058, 0.00369905,
                  0.00246271, 0.00172145, 0.0012504, 0.000936824, 0.000720019,
                  0.000565334, 0.000452006, 0.00036708, 0.00030218, 0.000251732,
                  0.000211924, 0.000180091, 0.000154331, 0.000133262, 0.000115861,
                  0.000101364, 0.0000891897])
VK2_8 = np.array([-0.0576957, 0., 0.00369053, 0.00148403, 0.000476966, 0.000171044,
                  0.0000725838, 0.0000351537, 0.000018759, 0.0000107737, 6.55467e-6,
                  4.17757e-6])
#cylinde for ly -= 9
VkAll_9 = np.zeros([12,12])
VkAll_9[0,:] = np.array([1.61969, 1.75847, 1.51951, 1.2485, 1.0394, 0.886327, 0.771114, 0.681538, 
                  0.610072, 0.551846, 0.503564, 0.462922])/3.937 #. 0.428265, 0.398378, 0.37235, 
                  #0.349485, 0.329245, 0.311206, 0.295028, 0.280441, 0.267221, 0.255186,
                  #0.244184, 0.234087])
VkAll_9[1,:] = np.array([0., 0.259392, 0.208305 , 0.1228, 0.0694229 , 0.0414582 , 0.0264649 , 0.017851 , 0.0125803,
                  0.00918539 , 0.00690438 , 0.005317 ])/3.937 #, 0.00417913, 0.00334263, 0.00271418, 0.00223303, 
                  #0.00185851, 0.0015627, 0.00132599, 0.00113437, 0.000977608, 0.000848144,
                  #0.000740299, 0.000649755])
VkAll_9[2,:] = np.array([0., 0., 0.00688152, 0.00366887, 0.0014533, 0.00057783, 0.000255901, 0.000126499, 0.0000682753, 0.0000394833, 0.0000241296, 0.0000154265])
VkAll_9[3,:] = np.array([0., 0., 0., 0.000312758, 0.000108588, 0.0000291495, 8.37891e-6, 2.84801e-6, 1.12663e-6, 5.00834e-7, 2.43624e-7, 1.27279e-7])
#cylinder for ly = 12
VkAll_12 = np.zeros([12,12])
VkAll_12[0,:] = np.array([0.320934, 0.384133, 0.372683, 0.334725, 0.294298, 0.25972, 0.231672, 
0.20881, 0.189843, 0.173856, 0.160213, 0.148449])
VkAll_12[1,:] = np.array([0., 0.0724515, 0.0784718, 0.0620882, 0.0437833, 0.030017, 0.0208383, 
0.0148232, 0.010797, 0.00802524, 0.00606777, 0.0046545])
VkAll_12[2,:] = np.array([0, 0., 0.0160811, 0.0136245, 0.0080551, 0.00403722, 
0.00181207, 0.000682583, 0.000115317, -0.000172564, -0.000318487, -0.000389945])
VkAll_12[3,:] = np.array([0, 0, 0., 0.00209261, 0.00112415, 0.000235222, 
                          -0.000185729, -0.000332923, -0.000366697, -0.000359974, -0.000340445, 
-0.000318093])
#torus for ly = 4
Vk0_4 = np.array([0.5796718, 0.32690444, 0.21313324, 0.161012,  0.13148597, 0.11285609,
         0.10040986, 0.09188379, 0.08607074, 0.08228929, 0.08015274, 0.07946119])
Vk1_4 = np.array([0, 1.01334690e-03, -9.02542319e-04, -9.53050786e-04,
         -8.63802178e-04, -7.78092606e-04, -7.10803456e-04, -6.60755064e-04,
         -6.24987215e-04, -6.01059246e-04, -5.87321594e-04, -5.82842372e-04])
#torus for ly = 8
Vk0_8 = np.array([8.61136038, 8.35002712, 6.69416235, 5.36293574, 4.49109225,
       3.91123899, 3.51118589, 3.23122926, 3.03768784, 2.91067149,
       2.83852972, 2.81512229])/8
Vk1_8 = np.array([0,  6.71835461e-01,  2.73394171e-01, -1.90576277e-02,
       -1.35149977e-01, -1.73747683e-01, -1.84836715e-01, -1.86281856e-01,
       -1.84630719e-01, -1.82479772e-01, -1.80905545e-01, -1.80341940e-01])/8
#Haldane-pseudo
VK0_4Hald = np.array([i**2*np.exp(-2*np.pi**2*i**2/4**2) for i in range(1,24)])/4
VK1_4Hald = np.array([(i**2-1)*np.exp(-2*np.pi**2*(i**2+1)/4**2) for i in range(1,24)])/4
VK2_4Hald = np.array([(i**2-4)*np.exp(-2*np.pi**2*(i**2+4)/4**2) for i in range(1,24)])/4
VK0_4Hald[4:]=0
VK1_4Hald[4:]=0
VK2_4Hald[3:]=0
VK0_4Hald
VkAllHald = np.zeros([Nphi//2,Nphi//2])
for i in K:
    for j in range(i,Nphi//2):
        VkAllHald[i,j] = ((j+1)**2-i**2)*np.exp(-2*np.pi**2*(i**2+(j+1)**2)/ly**2)/12
VkAllHald
ly

# Perturbation Theory plotting

In [ ]:
#Choose potentials
VK0 = VK0_4
VK0_reg = VK0_4reg
VK1 = VK1_4
U00 = U00_4
#VK0 = VK0_4Hald
#VK1 = VK1_4Hald

kmax = 21
#dipole interaction energies
DeltaVk = VK0[2:]-2*VK0[1:-1]+VK0[:-2]

#Zeroth order energies

def Ephs(s):
    krange = np.arange(3,kmax,3)
    E1 = np.sum(DeltaVk[krange[s-1:]-2])
    E2 = np.sum(krange[:s-1]*DeltaVk[krange[:s-1]-2]/3)
    return s*E1 + E2
Eph = Ephs(1)

#single dipole excitation energy
def Edip(d):
    jrange = np.arange(3,kmax,3)
    return  np.sum(VK0[np.abs(jrange-d)-1] - 2*VK0[jrange-1] + VK0[np.abs(jrange+d)-1])

#dipole pair energy
def Edipdip(p):
    return 2*Eph + DeltaVk[np.abs(p)-2]

#two dipole interaction energy
def Udipdip(d,p,q):
    Q = np.abs(q)
    P = np.abs(p)
    #NOTE: d is a VECTOR that points from p to q
    return np.sign(p*q)*(VK0[np.abs(d+Q-P)-1] - VK0[np.abs(d-P)-1] - VK0[np.abs(d+Q)-1] + VK0[np.abs(d)-1])

#Energy of a dipole pair in the P=1 sector
def Eph1dip(n,p):
    return Edipdip(p) - DeltaVk[np.abs(n)-2] + DeltaVk[np.abs(n+p)-2]

#energy of dipole-dipole-dipole-dipole pair, with mask
def Edipdipdipdip(n,k,p):
    U = np.zeros(np.size(p),dtype=float)
    isOutbound = (np.abs(n)>kmax)|(np.abs(n+p)>kmax)|(np.abs(n-k)>kmax)|(np.abs(n+p-k)>kmax)
    isOverlap = (n==k+1)|(n+p==k)|(n+p+1==0)|(n==0)
    P = p[(~isOutbound)&(~isOverlap)]
    if (np.abs(n-k)<kmax):
        U[(~isOutbound)&(~isOverlap)] = Deltadipdipdipdip(n,k,P)
    return Edipdip(p) + Edipdip(k) + U

def Uedipdipdipdip(m,n,k,p):
    Udipdipdipdip = Edipdipdipdip(n,k,p) - Edipdip(p) - Edipdip(k)
    isOutdipdip = (np.abs(Udipdipdipdip)<1e-14)
    isOutmn = (np.abs(m+k)>kmax)|(np.abs(n+p)>kmax)|(np.abs(n)>kmax)|(np.abs(m)>kmax)
    isAt0 = (m==0)|(n==0)|(m+k+1)==0|(n+p+1)==0
    return Udipdipdipdip

#Interaction of a dipole pair in the P = s sector
def Ephsdip(s,n,p):
    U = np.zeros(np.size(p),dtype=float)
    deltaU = np.zeros(np.size(p),dtype=float)
    isOutbound = ((np.abs(n)>kmax)|(np.abs(n+p)>kmax))
    isInDW = ((n<3*s)&(n>-1))|((n+p+1<3*s)&(n+p+1>-1))
    P = p[(~isOutbound)&(~isInDW)]
    #U[~isOutbound] = U[~isOutbound] + Edipdip(p[~isOutbound])
    for i in range(s):
        deltaU[(~isOutbound)&(~isInDW)] = Udipdip(n+P-P-3*i,1,1) + Udipdip(n+P-3*i,1,-1)
        U = U + deltaU
    return U + Edipdip(p)

def MaskPhsdip(s,n,p):
    isOutbound = ((np.abs(n)>kmax)|(np.abs(n+p)>kmax))
    isInDW = ((n<3*s)&(n>-1))|((n+p+1<3*s)&(n+p+1>-1))
    U = np.zeros(np.size(p),dtype=float)
    U[(~isOutbound)&(~isInDW)]=1
    return U
#Interaction energy of two dipole pairs (+)_0 (-)_k and (+)_n (-)_n+p
def Deltadipdipdipdip(n,k,p):
    return -DeltaVk[np.abs(n)-2] + DeltaVk[np.abs(n-k)-2] + DeltaVk[np.abs(n+p)-2] - DeltaVk[np.abs(n-k+p)-2]

#ground state total interaction energy per electron
def E0GS():
    krange = np.arange(3,kmax,3)
    return 2*np.sum(VK0_reg[krange-1])

#Interaction between dipole and an added electron
def Ue_dip(d,p):
    P = np.abs(p)
    return np.sign(p)*(VK0[np.abs(d+P)-1]-VK0[np.abs(d)-1])

#Inverted energy of dipole and electron ***AT n=0***
def Inv_Uedipdip(n,p,e):
    U = np.zeros(np.size(p),dtype=float)
    isOutbound = ((np.abs(n)>kmax)|(np.abs(n+p)>kmax))
    isIn0 = (n==0)|(n+p+1==0)
    P = p[(~isOutbound)&(~isIn0)]
    U[(~isOutbound)&(~isIn0)] = U[(~isOutbound)&(~isIn0)] + 1/(Edipdip(P) - e*Ue_dip(n,1) - e*Ue_dip(n+P,-1))
    return U

#dipole cluster states - excitation energy of s dipoles with positions given by n and moment given by p
def EphDips(n,p):
    E0 = len(n)*Ephs(1)
    for i in range(len(n)):
        for j in range(i+1,len(n)):
            if np.abs(n[i]-n[j])<kmax:
                E0 = E0 + Udipdip(n[j]-n[i],p[i],p[j])
    return E0

#removed electron state
def Eh():
    krange = np.arange(3,kmax,3)
    return -2*np.sum(VK0_reg[krange-1])-U00

#added electron state

def Ee():
    krange = np.arange(-kmax+2,kmax-1,3)
    return np.sum(VK0_reg[np.abs(krange-1)-1])

Nphi=24
#Eprint(E0('100100100100100010100100',VK0)-E0('100100100100100100100100',VK0))
Eph + VK0[10] - VK0[11]
EphDips([0,3],[1,1])
Eh()

In [ ]:
#First order correction
def Ephs_1(s,q):
    if (s==1):
        DE = 0
        k = 2
        while (k<kmax-1):
            DE = DE + 2*VK1[k]*np.cos(q*(k+1)/3)
            k = k+3
        return DE
    else:
        return 2*VK1[3*s-1]*np.cos(q)

#Second order corrections

#p-h state wrt ground state
def Eph1_2(s,q):
    krange = np.arange(-kmax+2,kmax-1,3)
    #diagonal part
    Egs2 = 2*np.sum(VK1[np.abs(krange)-1]**2/(Edipdip(krange)))
    EDip = 0
    for n in range(-kmax,kmax,3):
        EDip = EDip + np.sum(VK1[np.abs(krange)-1]**2/(Edipdip(krange)*Ephsdip(1,n,krange))*(-Edipdip(krange)+Ephsdip(1,n,krange)))
    EDip2 = -np.sum(VK1[np.abs(krange[~(krange==2)]-1)-1]**2/(Edip(2)+Udipdip(krange[~(krange==2)],2,-1)))
    #Off diagonal part
    Ephq = 0
    for n in range(-kmax,kmax,3):
        isOutbound = (np.abs(n+krange)>kmax)
        K = krange[~isOutbound]
        Ephq = Ephq + np.sum(2*VK1[np.abs(K)-1]*VK1[np.abs(n+K)-1]/Ephsdip(1,n,K)*MaskPhsdip(1,n,K))*np.cos(q*n/3)
    if (s==1):
        return Egs2 + EDip + EDip2 + Ephq
    else:
        return 0

#charged hole wrt ground state
def Eh_2():
    krange = np.arange(-kmax+2,kmax-1,3)
    Eh2 = 0
    Egs = np.sum(2*VK1[np.abs(krange)-1]**2/Edipdip(krange))
    for n in range(-kmax,kmax,3):
        isOutbound = ((np.abs(n)>kmax)|(np.abs(n+krange)>kmax))
        isIn0 = (n==0)|(n+krange+1==0)
        K = krange[(~isOutbound)&(~isIn0)]
        Eh2 = Eh2 + np.sum(VK1[np.abs(K)-1]**2*(- Ue_dip(n,1) - Ue_dip(n+K,-1))/(Edipdip(K) - Ue_dip(n,1) - Ue_dip(n+K,-1))/Edipdip(K))
    return Eh2 + Egs

#charged e wrt ground state
def Ee_2():
    krange = np.arange(-kmax+2,kmax-1,3)
    Ee2 = 0
    Egs = np.sum(2*VK1[np.abs(krange)-1]**2/Edipdip(krange))
    for n in range(-kmax,kmax,3):
        isOutbound = ((np.abs(n-1)>kmax)|(np.abs(n+krange-1)>kmax))
        isIn1 = (n-1==0)|(n+krange==0)
        K = krange[(~isOutbound)&(~isIn1)]
        Ee2 = Ee2 + np.sum(VK1[np.abs(K)-1]**2*(Ue_dip(n-1,1) + Ue_dip(n-1+K,-1))/(Edipdip(K) + Ue_dip(n-1,1) + Ue_dip(n-1+K,-1))/Edipdip(K))
    Eedip2 = -np.sum(VK1[np.abs(krange-1)-1]**2/(Eph+Ue_dip(krange-2,-1)))
    return Ee2 + Egs + Eedip2

#dipole-dipole uncharged excitations
def Edipdip_2(p,q):
    krange = np.arange(-kmax+2,kmax-1,3)
    #diagonal part
    Egs0 = VK1[np.abs(p)-1]**2/Edipdip(p)
    EgsDel = np.sum(2*VK1[np.abs(krange)-1]**2/Edipdip(krange))
    #dipdip to dipdip
    nrange = krange+1
    isnOutbound = (np.abs(nrange+p)>kmax)|(nrange==0)|(nrange+p+1==0)
    nn = nrange[~isnOutbound]
    EDipDip = 2*np.sum(VK1[np.abs(nn)-1]**2/(Edipdip(p) - Edipdip(p+nn))) #x2 to include correction due to both (+) and (-)
    #dipdip to 2dip2dip
    E2Dip2Dip = VK1[np.abs(p-2)-1]**2/(2*Edip(2)+Udipdip(p-1,2,-2))
    #dipdip to 2dipdipdip
    isKpOutbound = np.abs(krange-p)>kmax
    E2DipDipDip = -2*np.sum(VK1[np.abs(krange[~isKpOutbound]-1)-1]**2/(Edip(2) - Udipdip(p,1,-1) + Udipdip(krange[~isKpOutbound],-2,1) + Udipdip(krange[~isKpOutbound]-p,-1,-1)))
    #dipdip to dipdipdipdip
    EDipDipDipDip = 0
    for i in nrange:
        EDipDipDipDip = EDipDipDipDip + np.sum(VK1[np.abs(krange)-1]**2/(Edipdip(krange)*(Edipdipdipdip(i,p,krange)-Edipdip(p)))*(Edipdipdipdip(i,p,krange)-Edipdip(p)-Edipdip(krange)))
    #off-diagonal part
    EDipDipQ = 0
    for i in nn:
        EDipDipQ = EDipDipQ + 2*VK1[np.abs(i)-1]**2/(Edipdip(p)-Edipdip(p+i))*np.cos(q*i/3)
    EDipDipDipDipQ = 0
    P = np.array([p])
    for i in nrange:
        EDipDipDipDipQ = EDipDipDipDipQ + np.sum(VK1[np.abs(p)-1]**2/(Edipdip(p)*(Edipdipdipdip(i,p,P)-Edipdip(p)))*(Edipdipdipdip(i,p,P)-Edipdip(p)-Edipdip(P)))*np.cos(q*i/3)
    EgsQ = 2*VK1[np.abs(p)-1]**2/Edipdip(p)*np.cos(q*np.abs(p+1)/3)
    return Egs0+EgsDel+EDipDip+E2Dip2Dip+E2DipDipDip+EDipDipDipDip+EDipDipQ+EDipDipDipDipQ+EgsQ#(Edipdipdipdip(6,p,krange)-Edipdip(p)-Edipdip(krange))

def Ephs_2OffDiag(s,q):
    krange = np.arange(-kmax+2,kmax-1,3)
    nrange = (-kmax+3,kmax,3)
    #Domain wall start at -3 and extends to 3(s-1)-3
    srange = np.arange(-3,3*s,3)
    #First create the rightmost dipole at 3s
    isOutbound = (np.abs(3*s+krange)>kmax)
    isInDomain = (3*s+krange>-5)&(krange<-2)
    isNotEdge = (krange+1==-3*(s+1))
    #Out of domain first
    K = krange[(~isOutbound)&(~isInDomain)]
    EOut0 = 2*Eph + Udipdip(K,1,-1)
    for i in srange:
        EOut0 = EOut0 + Udipdip(3*s-i,1,1) + Udipdip(3*s+K-i,1,-1)
    EOutQ = -2*np.sum(VK1[np.abs(K)-1]*VK1[np.abs(K+3*(s+1))-1]/EOut0)*np.cos(q)
    #In domain states
    Kin = krange[(~isOutbound)&isInDomain&(~isNotEdge)]
    Ein0 = -Udipdip(0,1,1) + Udipdip(Kin+1,1,1)
    for i in srange:
        Ein0 = Ein0 + Udipdip(3*s+Kin+1-i,1,1) - Udipdip(3*s-i,1,1)
    EinQ = 2*np.sum(VK1[np.abs(Kin+1)-1]*VK1[np.abs(3*(s+1)+Kin+1)-1]/Ein0)*np.cos(q)
    #Double jump for s=1
    Edouble = 0
    if (s==1):
        for i in range(6,kmax,3):
            Edouble = Edouble + 2*VK1[i-1]**2/(Udipdip(3,1,1)-Udipdip(3+i,1,1))*np.cos(q*i/3)
        for i in range(9,kmax,3):
            Edouble = Edouble + 2*VK1[i-1]**2/(Udipdip(3,1,1)-Udipdip(3-i,1,1))*np.cos(q*i/3)
        for i in range(3,kmax,3):
            Edouble = Edouble + 4*VK1[i-1]*VK1[i+6-1]/(Udipdip(3,1,1) - Udipdip(i+3,1,1))*np.cos(q*i/3)
    return Edouble+EinQ+EOutQ
    

Edipdip_2(2,0)
#Deltadipdipdipdip(6,2,2)
Ephs_2OffDiag(1,0)
#Udipdip(3,1,-1)
#plt.plot(q,Ephs_1(3,q))
#VK1[5]
#Eph1_2(1,0)

In [ ]:
#VK0 = VK0_4Hald
#VK1 = VK1_4Hald

#STM spectra
kmax = 21
#electron AT n=1
def LDOSedipdip(n,k):
    isOutbound = (np.abs(n)>kmax)|(np.abs(n+k)>kmax)
    isAte = (n==0)
    if np.isscalar(k):
        LDOS = 1e-17
        E = 1e-17
        if ~(isOutbound|isAte):
            E = Edipdip(k) + Ue_dip(n-1,1)+ Ue_dip(n+k-1,-1)
            LDOS = VK1[np.abs(k)-1]**2*((E-Edipdip(k))/(E*Edipdip(k)))**2
        return E,LDOS
    else:
        LDOS = 1e-17*np.ones(np.size(p),dtype=float)
        E = 1e-17*np.ones(np.size(p),dtype=float)
        E[~(isOutbound|isAte)] = E[~(isOutbound|isAte)] + Edipdip(k) + Ue_dip(n-1,1) + Ue_dip(n+k-1,-1)
        LDOS[~(isOutbound|isAte)] = LDOS[~(isOutbound|isAte)]**2*((E[~(isOutbound|isAte)]-Edipdip(k))/(E[~(isOutbound|isAte)]*Edipdip(k)))**2
        return E,LDOS

def LDOSedip(p):
    if p==-1:
        return 0,0
    else:
        E = Eph + Ue_dip(-p-1,-1)
        LDOS = VK1[np.abs(p)-1]**2/E**2
        return E,LDOS

def LDOSe0dip(p):
    if p==-1:
        return 0,0
    else:
        E = Eph + Ue_dip(p-1,-1)
        LDOS = 2*VK1[np.abs(p)-1]**2/Edipdip(p)**2
        return E,LDOS

krangeSpec = np.arange(-kmax+2,kmax-1,3)
nrangeSpec = np.arange(-kmax,kmax,3)

SpecEe = []
SpecLDOSe = []
SpecEe_extra = []
SpecLDOSe_extra = []
Num = []
num = 0
for n in nrangeSpec:
    for k in krangeSpec:
        EL = LDOSedipdip(n,k)
        SpecEe.append(EL[0])
        SpecLDOSe.append(EL[1])
        Num.append([n,k])

for k in krangeSpec:
    EL = LDOSedip(k)
    SpecEe.append(EL[0])
    SpecLDOSe.append(EL[1])
    Num.append([k,101])
    EL = LDOSe0dip(k)
    SpecEe_extra.append(EL[0])
    SpecLDOSe_extra.append(EL[1])
    #Num.append([k,100])

def broadening(mu,sigma):
    return np.exp(-(E-mu)**2/(2*sigma**2))*1/np.sqrt(2*np.pi)



def LDOShdipdip(n,k):
    isOutbound = (np.abs(n)>kmax)|(np.abs(n+k)>kmax)
    isAte = (n==0)|(n+k+1==0)
    if np.isscalar(k):
        LDOS = 1e-17
        E = 1e-17
        if ~(isOutbound|isAte):
            E = Edipdip(k) - Ue_dip(n,1)- Ue_dip(n+k,-1)
            LDOS = VK1[np.abs(k)-1]**2*((E-Edipdip(k))/(E*Edipdip(k)))**2
        return E,LDOS
    else:
        LDOS = 1e-17*np.ones(np.size(p),dtype=float)
        E = 1e-17*np.ones(np.size(p),dtype=float)
        E[~(isOutbound|isAte)] = E[~(isOutbound|isAte)] + Edipdip(k) - Ue_dip(n,1) - Ue_dip(n+k,-1)
        LDOS[~(isOutbound|isAte)] = LDOS[~(isOutbound|isAte)]**2*((E[~(isOutbound|isAte)]-Edipdip(k))/(E[~(isOutbound|isAte)]*Edipdip(k)))**2
        return E,LDOS

def LDOSh0dip(p):
    if p==0:
        return 0,0
    else:
        E = Eph - Ue_dip(p,-1)
        LDOS = VK1[np.abs(p)-1]**2/Edipdip(p)**2
    return E,LDOS


SpecEh = []
SpecLDOSh = []
SpecEh_extra = []
SpecLDOSh_extra = []

for n in nrangeSpec:
    for k in krangeSpec:
        EL = LDOShdipdip(n,k)
        SpecEh.append(EL[0])
        SpecLDOSh.append(EL[1])

for k in krangeSpec:
    EL = LDOSh0dip(k)
    SpecEh_extra.append(EL[0])
    SpecLDOSh_extra.append(EL[1])

    
def broadening(mu,sigma):
    return np.exp(-(Egauss-mu)**2/(2*sigma**2))*1/np.sqrt(2*np.pi)
    
sd = 0.0001
LDOSe = np.zeros(5000)
LDOSh = np.zeros(5000)
Egauss = np.linspace(-0.25,0.25,5000)

#for i in range(len(SpecEh)):
#    plt.scatter(SpecEh[i],SpecLDOSh[i])
#plt.yscale('log')
for i in range(len(SpecEe)):
    LDOSe = LDOSe + broadening(SpecEe[i],sd)*SpecLDOSe[i]

LDOSe = LDOSe + broadening(0,sd)*(1-np.sum(SpecLDOSe))

for i in range(len(SpecEh)):
    LDOSh = LDOSh + broadening(SpecEh[i],sd)*SpecLDOSh[i]

LDOSh = LDOSh + broadening(0,sd)*(1-np.sum(SpecLDOSh))

plt.plot(Egauss+Ee(),LDOSe)
plt.plot(Egauss+Eh(),LDOSh)
plt.yscale('log')
plt.ylim(1e-14,0.5)
#plt.xlim(-0.15,0.15)
#SpecEe,SpecLDOSe
#np.argsort(SpecLDOSe)
#Num[111]
#SpecEe[197]
Eh()

In [ ]:
#Haldane only
V10 = VK0_4Hald[0]
V20 = VK0_4Hald[1]
V21 = VK1_4Hald[1]

#V20/V10 ~ 0.1, V21/V10~0.02, we treat both V21 and V20 as perturbation of the same order. 
#With just V10, multiple states are now degenerate
#1001101001, 0110101001, 1010011001, 1010010110 all has energy V10. 
#Perturbation matrix among these 4 states with H' = V20 + V21, ordered as above
VKMatrix = np.array([[V20,V21,V21,0],[V21,2*V20,0,0],[V21,0,V20,V21],[0,0,V21,2*V20]])
E1Hald, V1Hald = np.linalg.eigh(VKMatrix)

SpecEeHald = []
SpecLDOSeHald = []
for i in range(4):
    SpecEeHald.append(E1Hald[i])
    SpecLDOSeHald.append((V1Hald[0,i] - 2*V1Hald[1,i]*V21/V10)**2)

In [ ]:
q = np.linspace(0,2*np.pi,100)
krangePlot = [-10,-7,-4,2,5,8,11]
linerange = np.arange(0,1,1)
lineconst = np.ones(100)
kmax = 15
plt.plot(q,E0GS()*lineconst,'k-')
for k in krangePlot:
    plt.plot(q,E0GS()+Edipdip(k)+Edipdip_2(k,q),'k--')
    for n in krangePlot:
        plt.plot(q,lineconst*(E0GS()+Ee()+Ee_2()+Edipdip(k)+Ue_dip(n,1)+Ue_dip(n+k,-1)),color='orange',linestyle=':')
        plt.plot(q,lineconst*(E0GS()+Eh()+Eh_2()+Edipdip(k)-Ue_dip(n+1,1)-Ue_dip(n+k+1,-1)),color='purple',linestyle=':')
for s in range(1,5):
    plt.plot(q,E0GS()+Ephs(s)+Ephs_1(s,q),color=colors[np.abs(s)],linestyle=':')

plt.plot(q,lineconst*(Eh()+Eh_2()),color='purple',linestyle='-.')
plt.plot(q,lineconst*(Ee()+Ee_2()),color='orange',linestyle='dashdot')

#plt.ylim(0.01,0.05)

# Exact Diagonalization

In [ ]:
#BASIS SET UP

#Returns a list of the integer representation of all bitstring with nphi bytes and n ones
def bitList(n,nphi):
    if (n==0):
        return [0]
    else:
        Table = [(1<<n)-1]
        for i in range(n,nphi):
            SubTable = bitList(n-1,i)
            for j in SubTable:
                Table.append((1<<(i))+j)
        return Table

#Translates bitstring n by m sites to the left with PBC
def translate(n,m):
    nmax = (1<<Nphi)-1
    shift = m%Nphi
    nshift = (n<<shift)&nmax
    npbc = (n>>(Nphi-shift))
    return nshift|npbc

#courtesy of ChatGPT
def transCheck(Table):
    unique_bitstrings = set(Table)  # Use a set for efficient lookup
    final_bitstrings = set()  # Store unique representatives
    for bit in unique_bitstrings:
        # Check if any translation of 'bit' is already in final_bitstrings
        if any(translate(bit, i) in final_bitstrings for i in range(1, Nphi)):
            continue  # If a translation exists, skip this bitstring
        # Add 'bit' as the unique representative
        final_bitstrings.add(bit)    
    return list(final_bitstrings)  # Convert back to list

#Removes all bitstring that has a CM number different than cm (with PBC)
def dipCheck(Table,cm):
    dipTable = []
    PosTable = np.arange(Nphi,dtype=int)
    for bit in Table:
        arr = bitArray(bit)
        cmpos = np.sum(PosTable*arr)%Nphi
        if (cmpos==cm):
            #print('stay',bit)
            dipTable.append(bit)
    return dipTable

def TransSym(Table):
    out = []
    for x in Table:
        i = 1
        y = translate(x,1)
        while not(y==x):
            y = translate(y,1)
            i = i+1
        out.append(i)
    return out

In [ ]:
#HAMILTONIAN SET UP

#occupation number
def nocc(i,x):
    return (x>>(i%Nphi))&1

#creation and annihilation operators
def creat(i,x):
    return (1-(x>>(i%Nphi))&1)*((1<<(i%Nphi))^x)

def annih(i,x):
    return ((x>>(i%Nphi))&1)*((1<<(i%Nphi))^x)

#Coulomb/pair interactions between all particles, with potential V0
def HamDiag(Table, V0):
    Ham = []
    Ind = []
    i0 = 0
    for x in Table:
        E0 = 0
        for i in range(1,Nphi):
            y = translate(x,i)
            bits = bin(y&x)[2:]
            E0 = E0 + bits.count('1')*V0[min(i,Nphi-i)-1]
        Ham.append(E0/2)
        Ind.append(i0)
        i0 = i0 + 1
    return sparse.csr_matrix((Ham,(Ind,Ind)))

#Diagonal part, with zero for over-symmetric states on nonzero q
def HamDiagQ(Table, V0, q):
    Ham = []
    Ind = []
    sym = TransSym(Table)
    for i0,x in enumerate(Table):
        E0 = 0
        if (sym[i0]*q%Nphi==0):
            for i in range(1,Nphi):
                y = translate(x,i)
                bits = bin(y&x)[2:]
                E0 = E0 + bits.count('1')*V0[min(i,Nphi-i)-1]
        Ham.append(E0/2)
        Ind.append(i0)
        i0 = i0 + 1
    return sparse.csr_matrix((Ham,(Ind,Ind)))

#Two body interaction: moves particle at n+k and n+m by m sites in opposing direction
def squeeze(n,k,m,x):
    if nocc(n+k,x)&nocc(n+m,x)&(1-nocc(n,x))&(1-nocc(n+m+k,x)):
        return x^(1<<(n%Nphi))^(1<<((n+k+m)%Nphi))^(1<<((n+k)%Nphi))^(1<<((n+m)%Nphi))
    else:
        return 0  

#Creates the transformation matrix that block-diagonalizes the hamiltonian by momentum. Selects the block Hamiltonian with momentum q. Set d to 1 for most general results.
def TransU(Table,d,q):
    U = []
    IndOut = [len(Table)-1]
    table_index = {state: idx for idx, state in enumerate(Table)}
    tableT = transCheck(Table)
    IndIn = [len(tableT)-1]
    #Sym = TransSym(tableT)
    for i0,x in enumerate(tableT):
        n = 0
        #IndIn.append(i0)
        #IndOut.append(table_index[x])
        Ulist = [0]
        Indlist = [len(Table)-1]
        #Ulist.append(1)
        y = x
        while (n<Nphi):
            if(y in table_index):
                Ulist.append(np.exp(-2*1j*np.pi*q*n/Nphi))
                #IndIn.append(i0)
                Indlist.append(table_index[y])
            y = translate(y,d)
            n = n+d
        U_vect = sparse.csr_matrix((Ulist,(np.zeros(len(Indlist),dtype=int),Indlist)))
        norm = np.sqrt((U_vect@(U_vect.T).conj()).toarray()[0,0])
        if norm>(1e-14):
            U_vect = U_vect/norm
        else:
            U_vect = sparse.csr_matrix((1,len(Table)))
        U.append(U_vect)
    return sparse.vstack(U)

#Off diagonal element caused by squeeze(m)
def HamOffDiag(Table, m, Vm):
    Ham = [0]
    IndIn = [len(Table)-1]
    IndOut = [len(Table)-1]
    table_index = {state: idx for idx, state in enumerate(Table)}
    i0 = 0
    for x in Table:
        for i in range(Nphi):
            for j in range(m,min(int(Nphi/2),len(Vm))):
                y = squeeze(i,j+1,m,x)
                z = squeeze(i,j+1,-m,x)
                #print(i,j+1,z)
                if not(y==0):
                    Ham.append(Vm[j])
                    IndIn.append(i0)
                    IndOut.append(table_index[y])
                if not(z==0):
                    Ham.append(Vm[j])
                    IndIn.append(i0)
                    IndOut.append(table_index[z])
        i0 = i0+1
    return sparse.csr_matrix((Ham,(IndIn,IndOut)))

In [ ]:
#UTILITIES

#translates the eigenvector and the bit integer values
def eigvectBasis(V,Table):
    vect = []
    for i in range(len(V)):
        vect.append([V[i],Table[i]])
    return (vect)

#returns coulomb energy
def E0(m, V0):
    return HamDiag([int(m,2)],V0).toarray()

#return the configuration at index i
def bitAtIndex(Table,i):
    return [format(Table[i], f'0{Nphi}b')]

#convert the number/bitstring into a numpy array
def bitArray(m):
    divisor = 2**np.arange(Nphi,dtype=np.uint64)
    arr = (m//divisor)%2
    return np.flip(arr)

#calculatae the center of mass:
def cmPosX(m):
    PosTable = np.arange(Nphi,dtype=int)
    v = bitArray(m)
    return np.sum(PosTable*v)%Nphi

def printTable(Table):
    return print([format(x, f'0{Nphi}b') for x in Table])

In [ ]:
#build the basis for N=8 electrons, 001001001001001001001001 subspace
Nphi = 24
N = 8
BT = bitList(N,Nphi)
BT2 = dipCheck(BT,4)

#build the Hamiltonian
#%time H = HamDiagQ(BT2,VkAllHald[0,:],0)
#for i in range(1,5):
#    %time H = H + HamOffDiag(BT2,i,VkAllHald[i,:])
H = HamDiagQ(BT2,VK0_4,0) + HamOffDiag(BT2,1,VK1_4)

In [ ]:
#Diagonalize
#Eg, V = sparse.linalg.eigs(H,k=20,sigma=6,which='LM')
Eg, V = sparse.linalg.eigs(H,k=20,which='SM')
EGS_ED = Eg[0]
#Check the GS
print(Eg)
ind = np.argsort(np.abs(V[:,0]))[-1]
print(bitAtIndex(BT2,ind))
#np.sort(np.abs(V[:,0]))[-35:]
V[:,0][np.argsort(np.abs(V[:,0]))][-35:]
EGS_ED

In [ ]:
#diagonalization by dipole moment and momentum using TransU

Nphi = 24
N = 8
BT = bitList(N,Nphi)
SpecEDDip = [[] for i in range(Nphi//3)]
for d in range(Nphi//3):
    BT2 = dipCheck(BT,d)
    #H = HamDiagQ(BT2,VkAllHald[0,:],0)
    #for i in range(1,5):
    #    H = H + HamOffDiag(BT2,i,VkAllHald[i,:])
    H = HamDiagQ(BT2,VK0_4,0) + HamOffDiag(BT2,1,VK1_4)
    for i in range(Nphi//3):
        U = TransU(BT2,1,i)
        Ht = U@H@(U.T).conj()
        E = sparse.linalg.eigs(Ht,k=20,which='SM',return_eigenvectors=False) #sigma depends on the ground state energy
        SpecEDDip[d].append(E)

In [ ]:
#Diagonalization to find the P=0 states created by V_21, V_41, V_51, V_71
BT2 = dipCheck(BT,4)


SpecED2457Only = []
H = HamOffDiag(BT2,1,VK1_4,i)+HamDiagQ(BT2,VK0_4,0)
for i in range(Nphi//3):
    U = TransU(BT2,1,i)
    Ht = U@H@(U.T).conj()
    E2 = sparse.linalg.eigs(Ht,k=1,sigma=3.67,which='LM',return_eigenvectors=False)
    E4 = sparse.linalg.eigs(Ht,k=1,sigma=3.557,which='LM',return_eigenvectors=False)
    E5 = sparse.linalg.eigs(Ht,k=1,sigma=3.544,which='LM',return_eigenvectors=False)
    E7 = sparse.linalg.eigs(Ht,k=1,sigma=3.5375,which='LM',return_eigenvectors=False)
    SpecED2457Only.append([E2,E4,E5,E7])

In [ ]:
#build the basis, for N=9 electrons, 101001001001001001001001 subspace, i.e. add an electron to the i=0 state

bt = bitList(N+1,Nphi)
bt2 = dipCheck(bt,4)

#build the Hamiltonian
#h9 = HamDiagQ(bt2,VkAll_12[0,:],0)
#for i in range(1,5):
#   %time h9 = h9 + HamOffDiag(bt2,i,VkAll_12[i,:])
h9 = (HamOffDiag(bt2,1,VK1_4)+HamDiagQ(bt2,VK0_4,0))

In [ ]:
#Diagonalize the Hamiltonian
e2 = []
v2 = []
for i in range(3):
    u9 = TransU(bt2,1,i)
    h9t = u9@h9@(u9.T).conj()
    Eg9, v9 = sparse.linalg.eigs(h9t,k=330,which='SM')
    e2.append(Eg9)
    v2.append(v9)
V9 = u9.T@v2[2]

In [ ]:
#Add an electron at n=0 to the 8-electron ground state
BT = bitList(N,Nphi)
BT2 = dipCheck(BT,4)
#BT3 = transCheck(BT2)
H8 = (HamOffDiag(BT2,1,VK1_4)+HamDiag(BT2,VK0_4))
Eg8, V8 = sparse.linalg.eigs(H8,k=10,which='SM')

vect8 = eigvectBasis(V8[:,0],BT2)
vect8plus = []
for i in range(len(vect8)):
    vect8plus.append([vect8[i][0],creat(Nphi-1,vect8[i][1])])
vect8plus = np.array(vect8plus)

#Write vect8plus as a vector in the usual basis
vect8vect = np.zeros(len(V9[:,1]))
for i in range(len(vect8plus[:,1])):
    if not(vect8plus[i,1]==0):
        ind = bt2.index(int(vect8plus[i,1]))
        vect8vect[ind] = vect8plus[i,0]

In [ ]:
#Find the spectral representation coefficients

Coeff_Ee= []

for j in range(3):
    Eg9 = e2[j]
    V9 = u9.T@v2[j]
    for i in range(330):
        coeff = np.dot(V9[:,i],vect8vect)
        Coeff_Ee.append([np.abs(coeff)**2,Eg9[i]])


In [ ]:
#build the basis now for adding an electron to the i=23 site
bt_extra = bitList(N+1,Nphi)
bt2_extra = dipCheck(bt_extra,3)

#build the Hamiltonian
h9_extra = (HamOffDiag(bt2_extra,1,VK1_4)+HamDiagQ(bt2_extra,VK0_4,0))
#h9_extra = HamDiagQ(bt2_extra,VkAll[0,:],0)
#for i in range(1,6):
#    h9_extra = h9_extra + HamOffDiag(bt2_extra,i,VkAll[i,:])

In [ ]:
e2 = []
v2 = []
for i in range(3):
    u9 = TransU(bt2_extra,1,i)
    h9t = u9@h9_extra@(u9.T).conj()
    Eg9, v9 = sparse.linalg.eigs(h9t,k=200,which='SM')
    e2.append(Eg9)
    v2.append(v9)
V9 = u9.T@v2[0]

In [ ]:
Coeff_Ee_extra = []
bt_extra = bitList(N+1,Nphi)
bt2_extra = dipCheck(bt_extra,3)
BT = bitList(N,Nphi)
BT2 = dipCheck(BT,4)
H8 = (HamOffDiag(BT2,1,VK1_4)+HamDiagQ(BT2,VK0_4,0))
Eg8, V8 = sparse.linalg.eigs(H8,k=10,which='SM')

vect8 = eigvectBasis(V8[:,0],BT2)


vect8plus_extra = []
for i in range(len(vect8)):
    vect8plus_extra.append([vect8[i][0],creat(0,vect8[i][1])])
vect8plus_extra = np.array(vect8plus_extra)

#Write vect8plus as a vector in the usual basis
vect8vect_extra = np.zeros(len(V9[:,0]))
for i in range(len(vect8plus_extra[:,1])):
    if not(vect8plus_extra[i,1]==0):
        ind = bt2_extra.index(int(vect8plus_extra[i,1]))
        vect8vect_extra[ind] = vect8plus_extra[i,0]
for j in range(3):
    Eg9extra = e2[j]
    V9extra = u9.T@v2[j]
    for i in range(200):
        coeff = np.dot(V9extra[:,i],vect8vect_extra)
        Coeff_Ee_extra.append([np.abs(coeff)**2,Eg9extra[i]])


In [ ]:
#Calculate the spectrum for the N=7, P = 2 subspace (electron at i=0 removed). Note that there's only one q in this sector. 

bt = bitList(N-1,Nphi)
bt2 = dipCheck(bt,2)
h7 = (HamOffDiag(bt2,1,VK1)+HamDiag(bt2,VK0))

Eg7, V7 = sparse.linalg.eigs(h7,k=1000,sigma=2.534,which='LM')

In [ ]:
BT = bitList(N,Nphi)
BT2 = dipCheck(BT,4)
H8 = (HamOffDiag(BT2,1,VK1_4)+HamDiagQ(BT2,VK0_4,0))
Eg8, V8 = sparse.linalg.eigs(H8,k=10,which='SM')

vect8 = eigvectBasis(V8[:,0],BT2)
vect8min = []
for i in range(len(vect8)):
    vect8min.append([vect8[i][0],annih(Nphi-3,vect8[i][1])])
vect8min = np.array(vect8min)

#Write vect8plus as a vector in the usual basis
vect8minvect = np.zeros(len(V7[:,1]))
for i in range(len(vect8min[:,1])):
    if not(vect8min[i,1]==0):
        ind = bt2.index(int(vect8min[i,1]))
        vect8minvect[ind] = vect8min[i,0]


In [ ]:
Coeff_Eh = []
for i in range(500):
    coeff = np.dot(V7[:,i],vect8minvect)
    Coeff_Eh.append([np.abs(coeff)**2,Eg7[i]])

In [ ]:
#P=1 subspace; electron at i=1 removed.

bt_extra = bitList(N-1,Nphi)
bt2_extra = dipCheck(bt_extra,1)

#build the Hamiltonian
h7_extra = (HamOffDiag(bt2_extra,1,VK1)+HamDiagQ(bt2_extra,VK0,0))
#Eg7extra, V7extra = sparse.linalg.eigs(h7_extra,k=500,sigma=2.534,which='LM')
Eg7extra, V7extra = sparse.linalg.eigs(h7_extra,k=500,which='SM')

In [ ]:
bt_extra = bitList(N-1,Nphi)
bt2_extra = dipCheck(bt_extra,1)

vect8min_extra = []
for i in range(len(vect8)):
    vect8min_extra.append([vect8[i][0],annih(20,vect8[i][1])])
vect8min_extra = np.array(vect8min_extra)

#Write vect8plus as a vector in the usual basis
#V7 = u7.T@v3[0]
vect8vect_min = np.zeros(len(V7extra[:,1]))
for i in range(len(vect8min_extra[:,1])):
    if not(vect8min_extra[i,1]==0):
        ind = bt2_extra.index(int(vect8min_extra[i,1]))
        vect8vect_min[ind] = vect8min_extra[i,0]

Coeff_Eh_extra=[]
for i in range(500):
    coeff = np.dot(V7extra[:,i],vect8vect_min)
    Coeff_Eh_extra.append([np.abs(coeff)**2,Eg7extra[i]])

# Final Plotting

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'        # Options: 'sans-serif', 'serif', 'monospace', etc.
plt.rcParams['font.size'] = 14              # Global font size
plt.rcParams['axes.labelsize'] = 16         # Axes label size
plt.rcParams['axes.titlesize'] = 18         # Title size
plt.rcParams['legend.fontsize'] = 12        # Legend font size
plt.rcParams['xtick.labelsize'] = 12        # X-tick label size
plt.rcParams['ytick.labelsize'] = 12        # Y-tick label size
#plt.rcParams['axes.grid'] = True            # Enable grid by default
#plt.rcParams['lines.linewidth'] = 2
import matplotlib.gridspec as grid

In [ ]:
##FIGURE 2: Plotting the potentials
import matplotlib.gridspec as grid
plt.rcParams['axes.labelsize'] = 18         # Axes label size
plt.rcParams['axes.titlesize'] = 20         # Title size
plt.rcParams['legend.fontsize'] = 12        # Legend font size
plt.rcParams['xtick.labelsize'] = 16        # X-tick label size
plt.rcParams['ytick.labelsize'] = 16
K = range(1,12)
fig = plt.figure(figsize=(7,5))
plt.scatter(K,VK0[0:11],color='b')
plt.title('$V_{k0}$ and $V_{k1}$ coefficients for $L_y = 4l_B$')
plt.xlabel('$k$')
plt.ylabel('$V_{k0} ~(e^2/l_B)$')
plt.xticks(K)
ax_inset = plt.axes([0.5,0.5,0.35,0.35])
ax_inset.scatter(K,VK1[0:11],color='r')
ax_inset.set_xlabel('$k$')
ax_inset.set_ylabel('$V_{k1}~(e^2/l_B)$')
ax_inset.set_xticks(K)
ax_inset.set_yticks([0.000,0.001,0.002,0.003])
plt.savefig('fqhe_FigVkm.pdf')

In [ ]:
import matplotlib.gridspec as grid

## FIGURE 3: Plotting the zero-charge spectrum
plt.rcParams['axes.labelsize'] = 18         # Axes label size
plt.rcParams['axes.titlesize'] = 20         # Title size
plt.rcParams['legend.fontsize'] = 12        # Legend font size
plt.rcParams['xtick.labelsize'] = 14        # X-tick label size
plt.rcParams['ytick.labelsize'] = 14

q = np.linspace(0,2*np.pi,100)
krangePlot = [-10,-7,-4,2,5,8]
lineconst = np.ones(100)
kmax = 21
colors=['#000000','#0000FF','#00E1FA','#00DB2C','#FF0000']
markers=['o','x','+','d','s']
ymin= -0.01
ymax = 0.25
DeltaSize = VK0[10]-VK0[11]
#DeltaSize = VK0[11]-VK0[12]

fig = plt.figure(figsize=(10,5))
gs = grid.GridSpec(1,2,width_ratios=[1,1],height_ratios=[1])

ax0 = plt.subplot(gs[0])
#ax0.plot(q,E0GS()*lineconst,'k-',label='$P=0$')
ax0.scatter(0,E0GS(),color='k',marker='_',s=90)
ax0.plot(q,E0GS()+Edipdip(11)+Edipdip_2(11,q),'k--',label='$P=0$')
for k in krangePlot:
    ax0.plot(q,E0GS()+Edipdip(k)+Edipdip_2(k,q),'k--')
for s in range(1,5):
    ax0.plot(q,E0GS()+Ephs(s)+Ephs_1(s,q)+Eph1_2(s,q),color=colors[np.abs(s)],linestyle='--',label=f'$P={s}$')
ax0.set_title("Perturbation Theory")
ax0.set_xlabel("$2 \pi q/N$")
ax0.set_ylabel("Energy ($e^2/l_B$)")
ax0.set_ylim(E0GS()+ymin,E0GS()+ymax)
ax0.set_xticks([0,1,2,3,4,5,6])
plt.legend(ncol=3)

kmax = 12
q2 = np.linspace(0,7,100)
ax1 = plt.subplot(gs[1])
for k in krangePlot:
    ax1.plot(q,EGS_ED+Edipdip(k)+Edipdip_2(k,q)+1.5*DeltaSize,'k:')
for s in range(1,5):
    ax1.plot(q,EGS_ED+Ephs(s)+Ephs_1(s,q)+Eph1_2(s,q)+s*DeltaSize,color=colors[np.abs(s)],linestyle=':')
for d in range(0,Nphi//6+1):
    Ed = SpecEDDip[d%(Nphi//3)]
    for i,Ek in enumerate(Ed):
        for E in Ek:
            #plt.scatter(i,E,c=np.abs(d-4),cmap=cmap,vmin=0,vmax=4.5,s=35)
            ax1.scatter(2*np.pi*i/8,E,color=colors[abs(d-4)],marker='_',s=90)
ax1.set_title("Exact Diagonalization")
ax1.set_xlabel("$2 \pi q/N$")
ax1.set_ylabel("Energy ($e^2/l_B$)")
ax1.set_xlim(-0.4,6)
ax1.set_ylim(EGS_ED+ymin,EGS_ED+ymax)
ax1.set_xticks([0,1,2,3,4,5,6])

plt.tight_layout()
plt.savefig('fqhe_FigSpecFull.pdf')
plt.show()

In [ ]:
#FIGURE 4: Plotting the dispersion of the excitations:

plt.rcParams['xtick.labelsize'] = 18        # X-tick label size
plt.rcParams['ytick.labelsize'] = 18  
q = np.linspace(0,2*np.pi,100)
fig = plt.figure(figsize=(10,7))
gs = grid.GridSpec(4,2,width_ratios=[1,1],height_ratios=[1,1,1,1])

kmax=21
SizeCorr=[-0.0014,-0.00265,-0.004004,-0.005355]
#SizeCorr=[0.00015,0.000051,-0.004002,-0.005355]
#SizeCorr = [0,0,0,0]
colors=['#000000','#0000FF','#00E1FA','#00DB2C','#FF0000']
DeltaSize = VK0[10]-VK0[11]

ax0 = plt.subplot(gs[0,1])
Ed = SpecEDDip[5]
Yq = Ephs(1) + Ephs_1(1,q) + Eph1_2(1,q)
ax0.plot(q,Yq,color=colors[1])
for i,Ek in enumerate(Ed):
    ax0.scatter(i*(2*np.pi)/8,Ek[-1]-EGS_ED-DeltaSize-SizeCorr[0],color=colors[1],marker='o',s=25)
ax0.set_title('$P=1$',fontsize=22)
plt.setp(ax0.get_xticklabels(), visible=False)
#ax0.set_xlabel('q')

for d in range(2,4):
    Ed = SpecEDDip[np.abs(d-4)]
    Yq = Ephs(d) + Ephs_1(d,q) + Ephs_2OffDiag(d-1,q)
    ax = plt.subplot(gs[d-1,1],sharex = ax0)
    ax.plot(q,Yq,color=colors[d])
    for i,Ek in enumerate(Ed):
        ax.scatter(i*(2*np.pi)/8,Ek[-1]-EGS_ED-d*DeltaSize-SizeCorr[d-1],color=colors[d],marker='o',s=25)
    ax.set_title(f'$P={d}$',fontsize=22)
    plt.setp(ax.get_xticklabels(), visible=False)

Ed = SpecEDDip[0]
Yq = Ephs(4) + Ephs_1(4,q)
ax0l = plt.subplot(gs[3,1],sharex = ax0)
ax0l.plot(q,Yq,color=colors[4])
for i,Ek in enumerate(Ed):
    ax0l.scatter(i*(2*np.pi)/8,Ek[-1]-EGS_ED-4*DeltaSize-SizeCorr[3],color=colors[4],marker='o',s=25)
ax0l.set_title('$P=4$',fontsize=22)
ax0l.set_xlabel('$2\pi q/N$',fontsize=20)


SizeCorr2=[-0.00271,-0.00272,-0.00269,-0.00268]
ax1 = plt.subplot(gs[0,0])
Ed = SpecED2457Only
Ydipdip = Edipdip(2) + Edipdip_2(2,q)
ax1.plot(q,Ydipdip,color='k')
for i,Ek in enumerate(Ed):
    ax1.scatter(i*(2*np.pi)/8,Ek[0]-EGS_ED-2*DeltaSize-SizeCorr2[0],color='k',marker='o',s=25)
ax1.set_title('$P=0, k=2$',fontsize=22)
plt.setp(ax1.get_xticklabels(), visible=False)

for i,k in enumerate([-4,5]):
    ax = plt.subplot(gs[i+1,0])
    Ydipdip = Edipdip(k) + Edipdip_2(k,q)
    ax.plot(q,Ydipdip,color='k')
    for p,Ek in enumerate(Ed):
        ax.scatter(p*(2*np.pi)/8,Ek[i+1]-EGS_ED-2*DeltaSize-SizeCorr2[i+1],color='k',marker='o',s=25)
    ax.set_title(f'$P=0, k={k}$',fontsize=22)
    plt.setp(ax.get_xticklabels(), visible=False)

ax1l = plt.subplot(gs[3,0])
Ed = SpecED2457Only
Ydipdip = Edipdip(-7) + Edipdip_2(-7,q)
ax1l.plot(q,Ydipdip,color='k')
for i,Ek in enumerate(Ed):
    ax1l.scatter(i*(2*np.pi)/8,Ek[3]-EGS_ED-2*DeltaSize-SizeCorr2[3],color='k',marker='o',s=25)
ax1l.set_title('$P=0, k=-7$',fontsize=22)
ax1l.set_xlabel('$2\pi q/N$',fontsize=20)

plt.tight_layout()
plt.savefig('fqhe_FigDispersion.pdf')

In [ ]:
#FIGURE 5: Plotting the LDOS spectrum

def broadening(E,mu,sigma):
    return np.exp(-(E-mu)**2/(2*sigma**2))*1
sd = 0.0001
LDOSe = np.zeros(5000)
LDOSh = np.zeros(5000)
LDOSelong = np.zeros(10000)
LDOShlong = np.zeros(10000)
Egauss = np.linspace(-0.2,0.2,5000)
Elong = np.linspace(-0.2,0.5,10000)
LDOSeED = np.zeros(5000)
LDOShED = np.zeros(5000)
E0_e = 4.897
E0_h = 2.534


G = np.exp(-4*np.pi**2/4**2)
for i in range(len(SpecEe)):
    LDOSe = LDOSe + (1+G)*broadening(Egauss,SpecEe[i],sd)*SpecLDOSe[i]
    LDOSelong = LDOSelong + (1+G)*broadening(Elong,SpecEe[i],sd)*SpecLDOSe[i]
for i in range(len(SpecEe_extra)):
    LDOSe = LDOSe + G*broadening(Egauss,SpecEe_extra[i],sd)*SpecLDOSe_extra[i]
    LDOSelong = LDOSelong + (G)*broadening(Elong,SpecEe_extra[i],sd)*SpecLDOSe_extra[i]
LDOSe = LDOSe + (1+G)*broadening(Egauss,0,sd)*(1-np.sum(SpecLDOSe))
LDOSelong = LDOSelong + (1+G)*broadening(Elong,0,sd)*(1-np.sum(SpecLDOSe))

for i in range(len(SpecEh)):
    LDOSh = LDOSh + (1)*broadening(Egauss,SpecEh[i],sd)*SpecLDOSh[i]
    LDOShlong = LDOShlong + (1)*broadening(Elong,SpecEh[i],sd)*SpecLDOSh[i]
for i in range(len(SpecEh_extra)):
    LDOSh = LDOSh + 2*G*broadening(Egauss,SpecEh_extra[i],sd)*SpecLDOSh_extra[i]
    LDOShlong = LDOShlong + 2*G*broadening(Elong,SpecEh_extra[i],sd)*SpecLDOSh_extra[i]
LDOSh = LDOSh + (1)*broadening(Egauss,0,sd)*(1-np.sum(SpecLDOSh))
LDOShlong = LDOShlong + (1)*broadening(Elong,0,sd)*(1-np.sum(SpecLDOSh))

for ei in Coeff_Ee:
    if (ei[0]>1e-17)&(ei[0]<1e-2):
        LDOSeED = LDOSeED + (1+G)*broadening(Egauss+E0_e+1.5*DeltaSize,ei[1],sd)*ei[0]
    elif (ei[0]>1e-2):
        LDOSeED = LDOSeED + (1+G)*broadening(Egauss+E0_e,ei[1],sd)*ei[0]
for ei in Coeff_Ee_extra:
    if (ei[0]>1e-17)&(ei[0]<1e-2):
        LDOSeED = LDOSeED + G*broadening(Egauss+E0_e+1.5*DeltaSize,ei[1],sd)*ei[0]
for ei in Coeff_Eh:
    if (ei[0]>1e-17)&(ei[0]<1e-2):
        LDOShED = LDOShED + (1)*broadening(Egauss+E0_h+1.5*DeltaSize,ei[1],sd)*ei[0]
    elif (ei[0]>1e-2):
        LDOShED = LDOShED + (1)*broadening(Egauss+E0_h,ei[1],sd)*ei[0]
for ei in Coeff_Eh_extra:
    if (ei[0]>1e-17)&(ei[0]<1e-2):
        LDOShED = LDOShED + 2*G*broadening(Egauss+E0_h+1.5*DeltaSize,ei[1],sd)*ei[0]

plt.rcParams['axes.labelsize'] = 20         # Axes label size
plt.rcParams['axes.titlesize'] = 22        # Title size
plt.rcParams['legend.fontsize'] = 12        # Legend font size
plt.rcParams['xtick.labelsize'] = 20       # X-tick label size
plt.rcParams['ytick.labelsize'] = 20   

fig = plt.figure(figsize=(12,12))
gs = grid.GridSpec(2,2)

ax0 = plt.subplot(gs[1,1])
ax0.plot(Egauss+Ee(),LDOSe,'b-')
ax0.plot(Egauss+Ee(),LDOSeED,'k--')
#plt.plot(Egauss,LDOSh)
ax0.set_yscale('log')
ax0.set_ylim(1e-7,10)
ax0.set_xlim(-0.15+Ee(),0.15+Ee())
ax0.set_xlabel('Energy ($e^2/l_B$)')
ax0.set_ylabel(r'LDOS($x=\frac{2\pi l_B^2}{L_y},E$)')
ax0.set_title('Electron tunneling spectra')
ax_inset = plt.axes([0.68,0.41,0.27,0.05])
ax_inset.plot(Elong+Ee(),LDOSelong,'b-')
ax_inset.set_yscale('log')
ax_inset.set_yticklabels([])
ax_inset.set_ylim(1e-8,1)
#ax_inset.set_xlabel('Energy ($e^2/l_B$)',fontsize=10)
#ax_inset.set_xlabel('$k$')
#ax_inset.set_ylabel('$V_{k1}~(e^2/l_B)$')

ax1 = plt.subplot(gs[0,1])
ax1.plot(Egauss+Eh(),LDOSh,'r-')
ax1.plot(Egauss+Eh(),LDOShED,'k--')
#plt.plot(Egauss,LDOSh)
ax1.set_yscale('log')
ax1.set_ylim(1e-7,10)
ax1.set_xlim(-0.15+Eh(),0.15+Eh())
ax1.set_xlabel('Energy ($e^2/l_B$)')
ax1.set_title('Hole tunneling spectra')
ax1.set_ylabel(r'LDOS($x=0,E$)')
ax_inset1 = plt.axes([0.68,0.9,0.27,0.05])
ax_inset1.plot(Elong+Eh(),LDOShlong,'r-')
ax_inset1.set_yscale('log')
ax_inset1.set_xlim([0.75,1.1])
ax_inset1.set_yticklabels([])
ax_inset1.set_ylim(1e-8,1)
plt.tight_layout()
#plt.show()
#plt.savefig('fqhe_FigSpectra.pdf')

ax2 = plt.subplot(gs[1,0])
for i in range(len(SpecEe)):
    if (SpecLDOSe[i]>1e-6):
        #size = 100+12*np.log10((1+G)*SpecLDOSe[i])
        size = (100000*(1+G)*SpecLDOSe[i])
        op = 1+0.12*np.log10((1+G)*SpecLDOSe[i])
        ax2.scatter(1,SpecEe[i]+Ee(),color='b',marker='_',s=size,alpha=op)
        ax2.scatter(2,SpecEe[i]+Ee(),color='b',marker='_',s=size,alpha=op)
    if (2*G*SpecLDOSe[i]>1e-6):
        #size = 100+12*np.log10(2*G*SpecLDOSe[i])
        size = (100000*(2*G)*SpecLDOSe[i])
        op = 1+0.12*np.log10(2*G*SpecLDOSe[i])
        ax2.scatter(0,SpecEe[i]+Ee(),color='b',marker='_',s=size,alpha=op)
        ax2.scatter(3,SpecEe[i]+Ee(),color='b',marker='_',s=size,alpha=op)
        #print(size)
ax2.scatter(1,Ee(),color='b',linewidth=3,marker='_',edgecolor='b',s=1500)
ax2.scatter(2,Ee(),color='b',linewidth=3,marker='_',edgecolor='b',s=1500)
ax2.scatter(3,Ee(),color='b',linewidth=3,marker='_',edgecolor='b',s=1500*G)
ax2.scatter(0,Ee(),color='b',linewidth=3,marker='_',edgecolor='b',s=1500*G)
for i in range(len(SpecEe_extra)):
    if (SpecLDOSe_extra[i]>1e-6):
        #size = 100+12*np.log10(SpecLDOSe_extra[i])
        size = 100000*SpecLDOSe_extra[i]
        op = 1+0.12*np.log10(SpecLDOSe_extra[i])
        ax2.scatter(0,SpecEe_extra[i]+Ee(),color='b',marker='_',s=size,alpha=op)
        ax2.scatter(3,SpecEe_extra[i]+Ee(),color='b',marker='_',s=size,alpha=op)
    if (G*SpecLDOSe_extra[i]>1e-6):
        #size = 100+12*np.log10(G*SpecLDOSe_extra[i])
        size = 100000*G*SpecLDOSe_extra[i]
        op = 1+0.12*np.log10(G*SpecLDOSe_extra[i])
        ax2.scatter(1,SpecEe_extra[i]+Ee(),color='b',marker='_',s=size,alpha=op)
        ax2.scatter(2,SpecEe_extra[i]+Ee(),color='b',marker='_',s=size,alpha=op)
        #print(size)
ax2.set_ylim(-0.3,0.15)
#ax3.set_xlabel('$x$ Position')
ax2.set_title('Position and energy dependence of electron LDOS')
ax2.set_ylabel('Energy ($e^2/l_B$)')
ax2.set_xlabel('$x$ Position')
ax2.set_xticks([0,1,2,3])
ax2.set_xticklabels([0,r'$\frac{2 \pi l_B^2}{L_y}$',r'$\frac{4 \pi l_B^2}{L_y}$',r'$\frac{6 \pi l_B^2}{L_y}$'],fontsize=20)


ax3 = plt.subplot(gs[0,0])
ax3.scatter(0,Eh(),color='r',linewidth=3,marker='_',edgecolor='r',s=1500)
ax3.scatter(3,Eh(),color='r',linewidth=3,marker='_',edgecolor='r',s=1500)
ax3.scatter(1,Eh(),color='r',linewidth=3,marker='_',edgecolor='r',s=1500*G)
ax3.scatter(2,Eh(),color='r',linewidth=3,marker='_',edgecolor='r',s=1500*G)
for i in range(len(SpecEh)):
    if (SpecLDOSh[i]>1e-6):
        #size = 100+12*np.log10((1+G)*SpecLDOSe[i])
        size = (100000*(1)*SpecLDOSh[i])
        op = 1+0.12*np.log10((1)*SpecLDOSh[i])
        ax3.scatter(0,SpecEh[i]+Eh(),color='r',marker='_',s=size,alpha=op)
        ax3.scatter(3,SpecEh[i]+Eh(),color='r',marker='_',s=size,alpha=op)
    if (G*SpecLDOSh[i]>1e-6):
        #size = 100+12*np.log10(2*G*SpecLDOSe[i])
        size = (100000*(G)*SpecLDOSh[i])
        op = 1+0.12*np.log10(G*SpecLDOSh[i])
        ax3.scatter(0,SpecEh[i]+Eh(),color='r',marker='_',s=size,alpha=op)
        ax3.scatter(3,SpecEh[i]+Eh(),color='r',marker='_',s=size,alpha=op)
for i in range(len(SpecEh_extra)):
    if ((1+G)*SpecLDOSh_extra[i]>1e-6):
        #size = 100+12*np.log10(SpecLDOSe_extra[i])
        size = 100000*(1+G)*SpecLDOSh_extra[i]
        op = 1+0.12*np.log10((1+G)*SpecLDOSh_extra[i])
        ax3.scatter(1,SpecEh_extra[i]+Eh(),color='r',marker='_',s=size,alpha=op)
        ax3.scatter(2,SpecEh_extra[i]+Eh(),color='r',marker='_',s=size,alpha=op)
    if (2*G*SpecLDOSh_extra[i]>1e-6):
        #size = 100+12*np.log10(G*SpecLDOSe_extra[i])
        size = 100000*G*2*SpecLDOSh_extra[i]
        op = 1+0.12*np.log10(2*G*SpecLDOSh_extra[i])
        ax3.scatter(0,SpecEh_extra[i]+Eh(),color='r',marker='_',s=size,alpha=op)
        ax3.scatter(3,SpecEh_extra[i]+Eh(),color='r',marker='_',s=size,alpha=op)
ax3.set_ylim(0.8,1.12)
#ax3.set_title('Position and energy dependence of LDOS')
ax3.set_ylabel('Energy ($e^2/l_B$)')
ax3.set_xticks([0,1,2,3])
ax3.set_xlabel('$x$ Position')
ax3.set_xticklabels([],fontsize=20)
ax3.set_title('Position and energy dependence of hole LDOS')
ax3.set_xticklabels([0,r'$\frac{2 \pi l_B^2}{L_y}$',r'$\frac{4 \pi l_B^2}{L_y}$',r'$\frac{6 \pi l_B^2}{L_y}$'],fontsize=20)
#ax2.xaxis.set_tick_params(labelsize=16)

plt.tight_layout()
plt.savefig('fqhe_FigSpectra.pdf')

In [ ]:
#FIGURE 6: Plotting the ED LDOS spectrum for thicker tori. Need to rerun the ED code for different potentials.

plt.rcParams['axes.labelsize'] = 20         # Axes label size
plt.rcParams['axes.titlesize'] = 22        # Title size
plt.rcParams['legend.fontsize'] = 12        # Legend font size
plt.rcParams['xtick.labelsize'] = 20        # X-tick label size
plt.rcParams['ytick.labelsize'] = 20

fig = plt.figure(figsize=(10,8))

def broadening(E,mu,sigma):
    return np.exp(-(E-mu)**2/(2*sigma**2))*1
sd = 0.0001
sd2 = 0.01

ax = plt.subplot(3,2,4)
Egauss = np.linspace(7.62,7.82,50000)
LDOS = np.zeros(50000)
LDOS2 = np.zeros(50000)
for ei in Coeff_Ee_04:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
        LDOS2 = LDOS2 + broadening(Egauss,ei[1],sd2)*ei[0]
ax.plot(Egauss,LDOS2/np.sqrt(2*np.pi),'b')
ax.plot(Egauss,LDOS,'k')
ax.set_xlim([7.62,7.82])
ax.set_ylim(bottom=0)
ax.set_title(r'$\alpha=0.4$')

ax = plt.subplot(3,2,1)
Egauss = np.linspace(6.89,7.09,50000)
LDOS = np.zeros(50000)
LDOS2 = np.zeros(50000)
for ei in Coeff_Ee_025:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
        LDOS2 = LDOS2 + broadening(Egauss,ei[1],sd2)*ei[0]
ax.plot(Egauss,LDOS2/np.sqrt(2*np.pi),'b')
ax.plot(Egauss,LDOS,'k')
ax.set_xlim([6.89,7.09])
ax.set_ylim(bottom=0)
ax.set_title(r'$\alpha=0.25$')

ax = plt.subplot(3,2,2)
Egauss = np.linspace(7.21,7.41,50000)
LDOS = np.zeros(50000)
LDOS2 = np.zeros(50000)
for ei in Coeff_Ee_03:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
        LDOS2 = LDOS2 + broadening(Egauss,ei[1],sd2)*ei[0]
ax.plot(Egauss,LDOS2/np.sqrt(2*np.pi),'b')
ax.plot(Egauss,LDOS,'k')
ax.set_xlim([7.21,7.41])
ax.set_ylim(bottom=0)
ax.set_title(r'$\alpha=0.3$')

ax = plt.subplot(3,2,5)
Egauss = np.linspace(7.9,8.1,50000)
LDOS = np.zeros(50000)
LDOS2 = np.zeros(50000)
for ei in Coeff_Ee_055:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
        LDOS2 = LDOS2 + broadening(Egauss,ei[1],sd2)*ei[0]
ax.plot(Egauss,LDOS2/np.sqrt(2*np.pi),'b')
ax.plot(Egauss,LDOS,'k')
ax.set_xlim([7.9,8.1])
ax.set_ylim(bottom=0)
ax.set_xlabel('Energy $(e^2/l_B)$')
ax.set_title(r'$\alpha=0.55$')

ax = plt.subplot(3,2,3)
Egauss = np.linspace(7.45,7.65,50000)
LDOS = np.zeros(50000)
LDOS2 = np.zeros(50000)
for ei in Coeff_Ee_035:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
        LDOS2 = LDOS2 + broadening(Egauss,ei[1],sd2)*ei[0]
ax.plot(Egauss,LDOS2/np.sqrt(2*np.pi),'b')
ax.plot(Egauss,LDOS,'k')
ax.set_xlim([7.45,7.65])
ax.set_ylim(bottom=0)
ax.set_title(r'$\alpha=0.35$')

ax = plt.subplot(3,2,6)
Egauss = np.linspace(8,8.25,50000)
LDOS = np.zeros(50000)
LDOS2 = np.zeros(50000)
for ei in Coeff_Ee_07:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
        LDOS2 = LDOS2 + broadening(Egauss,ei[1],sd2)*ei[0]
ax.plot(Egauss,LDOS2/np.sqrt(2*np.pi),'b')
ax.plot(Egauss,LDOS,'k')
ax.set_xlim([8,8.25])
ax.set_ylim(bottom=0)
ax.set_xlabel('Energy $(e^2/l_B)$')
ax.set_title(r'$\alpha=0.7$')

plt.tight_layout()
#plt.savefig('fqhe_FigSpecAlpha2.pdf')

In [ ]:
##Same thing

fig = plt.figure(figsize=(10,6))

def broadening(E,mu,sigma):
    return np.exp(-(E-mu)**2/(2*sigma**2))*1
sd = 0.01

ax = plt.subplot(3,2,1)
Egauss = np.linspace(6.87,7.07,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_025:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([6.87,7.07])
ax.set_title(r'$\alpha=0.25$')

ax = plt.subplot(3,2,2)
Egauss = np.linspace(7.05,7.25,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_0275:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.05,7.25])
ax.set_title(r'$\alpha=0.275$')

ax = plt.subplot(3,2,3)
Egauss = np.linspace(7.2,7.4,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_03:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.2,7.4])
ax.set_ylim(bottom=0)
ax.set_title(r'$\alpha=0.3$')

ax = plt.subplot(3,2,4)
Egauss = np.linspace(7.35,7.55,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_0325:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.35,7.55])
ax.set_title(r'$\alpha=0.325$')

ax = plt.subplot(3,2,5)
Egauss = np.linspace(7.45,7.65,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_035:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.45,7.65])
ax.set_title(r'$\alpha=0.35$')

ax = plt.subplot(3,2,6)
Egauss = np.linspace(7.6,7.8,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_04:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.6,7.8])
ax.set_title(r'$\alpha=0.4$')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10,6))

def broadening(E,mu,sigma):
    return np.exp(-(E-mu)**2/(2*sigma**2))*1
sd = 0.01

ax = plt.subplot(3,2,2)
Egauss = np.linspace(6.82,7.02,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_Cyl_025:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([6.82,7.02])
ax.set_title(r'$\alpha=0.25$')

ax = plt.subplot(3,2,1)
Egauss = np.linspace(6.3,6.5,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_Cyl_02:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([6.3,6.5])
ax.set_title(r'$\alpha=0.2$')

ax = plt.subplot(3,2,3)
Egauss = np.linspace(7.2,7.4,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_Cyl_03:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.2,7.4])
ax.set_title(r'$\alpha=0.3$')

ax = plt.subplot(3,2,6)
Egauss = np.linspace(8.1,8.45,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_Cyl_054:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([8.1,8.45])
ax.set_title(r'$\alpha=0.55$')

ax = plt.subplot(3,2,4)
Egauss = np.linspace(7.45,7.65,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_Cyl_035:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.47,7.65])
ax.set_title(r'$\alpha=0.35$')

ax = plt.subplot(3,2,5)
Egauss = np.linspace(7.68,7.88,50000)
LDOS = np.zeros(50000)
for ei in Coeff_Ee_Cyl_04:
    if (ei[0]>1e-17):
        LDOS = LDOS + broadening(Egauss,ei[1],sd)*ei[0]
ax.plot(Egauss,LDOS)
ax.set_xlim([7.68,7.88])
ax.set_title(r'$\alpha=0.4$')

plt.tight_layout()

In [ ]:
#Plotting the energies of the charged excitations

fig = plt.figure(figsize=(5,6))

plt.plot(q,E0GS()*lineconst,'k-',linewidth=2)
for k in krangePlot:
    for n in krangePlot:
        plt.plot(q,lineconst*(E0GS()+Ee()+Ee_2()+Edipdip(k)+Ue_dip(n,1)+Ue_dip(n+k,-1)),color='purple',linestyle='-',linewidth=2+0.2*np.log(VK1[np.abs(k)-1]))
        plt.plot(q,lineconst*(E0GS()+Eh()+Eh_2()+Edipdip(k)-Ue_dip(n+1,1)-Ue_dip(n+k+1,-1)),color='orange',linestyle='-',linewidth=2+0.2*np.log(VK1[np.abs(k)-1]))
plt.plot(q,lineconst*(Eh()+Eh_2()),color='r',linestyle='-',linewidth=2)
plt.plot(q,lineconst*(Ee()+Ee_2()),color='b',linestyle='-',linewidth=2)
plt.xticks(visible=False)
plt.ylabel('Energy')
Eh()

# Scratchpad